In [64]:
import config
import json, csv, pprint, time, xlsxwriter
import numpy as np
import pandas as pd

from datetime import datetime
from binance.client import Client
from talib import RSI, PLUS_DI, DX, CCI, ADX, MINUS_DI,ATR
from binance.websockets import BinanceSocketManager
from binance.enums import *
import bb
client = Client(config.API_KEY, config.API_SECRET)
import talib
import os

In [3]:
klines = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_15MINUTE, "1 day ago UTC")
ohlc = bb.get_ohlc_his(klines)
open = ohlc[0]
high = ohlc[0]
low = ohlc[0]
close = ohlc[0]

In [5]:
integer = talib.CDLMORNINGSTAR(open, high, low, close, penetration=0)
print(integer)

In [7]:
ADX = talib.ADX(high, low, close, timeperiod=14)
DX = talib.DX(high,low,close,timeperiod=14)
PDI = talib.PLUS_DI(high, low, close, timeperiod=14)

In [8]:
print(DX)
print('----')
print(PDI)

[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan 24.9266682  32.67967879 32.09790454 29.34930556
 30.37741237 31.59620107  5.91994187  8.54710538 55.54569571 81.1460828
 63.51240543 72.72414728 64.48857857 42.37945861 47.55614369 48.94681833
 35.67192229 40.60472595 36.97715785 39.98147236 38.60225129 39.49205457
 44.97796472 25.65175599 22.41092363 19.690638    9.63158229 12.64981934
 15.65607254 15.57859489 11.3744178  17.76705349  5.99861351  2.89987913
  8.32785558 10.19185041  8.24721878 11.70721651 14.40080841 11.49085253
 21.1389004  35.78456166 29.74031463 37.22347678 35.19522101 40.27302553
 36.97114042 31.46610394 37.88922653 35.5190564  45.94855196 57.64364578
 50.6838834  54.84708007 59.90904043 53.7862373  58.22899733 45.32362702
 37.89746053 23.2247321  26.59931098 23.60790386 14.05344637 19.34235587
 20.44120717  7.5474244  12.79231762 19.19739093 27.

In [9]:
def isSupport(df,i):
    support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] \
    and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]

    return support

def isResistance(df,i):
    resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] \
    and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2] 

    return resistance


def isSupportRSI(df,i):
    support = df['RSI'][i] < df['RSI'][i-1]  and df['RSI'][i] < df['RSI'][i+1] \
    and df['RSI'][i+1] < df['RSI'][i+2] and df['RSI'][i-1] < df['RSI'][i-2]

    return support

def isResistanceRSI(df,i):
    resistance = df['RSI'][i] > df['RSI'][i-1]  and df['RSI'][i] > df['RSI'][i+1] \
    and df['RSI'][i+1] > df['RSI'][i+2] and df['RSI'][i-1] > df['RSI'][i-2] 

    return resistance

In [10]:
klines = client.get_historical_klines("SUSHIUSDT", Client.KLINE_INTERVAL_15MINUTE, "03 Feb, 2021")

In [11]:
df = pd.DataFrame(klines, columns=config.FIELD_HISTORICAL)

In [12]:
df.head()

,Opem Time,Open,High,Low,Close,Volume,Close Time,QA Volume,Number of Trades,TBBA Volume,TBQA Volume,Ignore
0,1612310400000,12.64300000,12.82000000,12.60100000,12.77300000,99851.82800000,1612311299999,1267688.71760800,1735,44941.42000000,570472.16350300,0
1,1612311300000,12.76000000,12.81700000,12.68400000,12.73100000,56993.78700000,1612312199999,725939.65179800,1161,32914.77300000,419057.71636300,0
2,1612312200000,12.73100000,12.74800000,12.60300000,12.61400000,67620.61400000,1612313099999,856757.39279600,1417,37990.93500000,481249.67021700,0
3,1612313100000,12.61400000,12.67600000,12.52600000,12.55600000,87331.60100000,1612313999999,1099597.72124900,1569,46110.27300000,580630.28573700,0
4,1612314000000,12.54100000,12.67100000,12.45300000,12.63100000,82982.56300000,1612314899999,1039545.58916200,1592,33741.17700000,422867.84154900,0


In [13]:
dohlc = df[['Opem Time','Open','High','Low','Close','Volume']].copy().set_index('Opem Time')

In [14]:
from datetime import datetime
dohlc.index.name = 'Date'
dohlc.head()
dates = dohlc.index
readable_dates = []
for date in dates : 
    ts = date/1000
    rd =  datetime.utcfromtimestamp(ts)
    readable_dates.append(rd)
readable_dates

dohlc.set_index(pd.DatetimeIndex(readable_dates), inplace=True)

In [15]:
rsi = talib.RSI(dohlc['Close'], 14)

In [16]:
ohlc_rsi = dohlc
ohlc_rsi['RSI'] = rsi

In [17]:
ohlc_rsi.dropna(inplace=True)

In [18]:
ohlc_rsi.head()


,Open,High,Low,Close,Volume,RSI
2021-02-03 03:30:00,12.81500000,12.88600000,12.53400000,12.71800000,173047.06900000,47.744053
2021-02-03 03:45:00,12.71900000,12.73100000,12.65100000,12.66600000,60173.90000000,45.647059
2021-02-03 04:00:00,12.66200000,12.78600000,12.65700000,12.78300000,75113.59500000,50.875184
2021-02-03 04:15:00,12.78500000,12.90400000,12.78000000,12.79600000,88620.23200000,51.434164
2021-02-03 04:30:00,12.79600000,12.87500000,12.76600000,12.82900000,48473.90700000,52.899299


In [19]:
levels = []
for i in range(2,ohlc_rsi.shape[0]-2):
    if isSupportRSI(ohlc_rsi,i):
        levels.append((i,ohlc_rsi['RSI'][i]))
    elif isResistance(ohlc_rsi,i):
        levels.append((i,ohlc_rsi['RSI'][i]))

In [20]:
def isSupport(df,i):
    support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] \
    and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]

    return support

def isResistance(df,i):
    resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] \
    and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2] 

    return resistance


levels = []
for i in range(2,df.shape[0]-2):
    if isSupport(df,i):
        levels.append((i,df['Low'][i]))
    elif isResistance(df,i):
        levels.append((i,df['High'][i]))

In [21]:
dohlc.head()

,Open,High,Low,Close,Volume,RSI
2021-02-03 03:30:00,12.81500000,12.88600000,12.53400000,12.71800000,173047.06900000,47.744053
2021-02-03 03:45:00,12.71900000,12.73100000,12.65100000,12.66600000,60173.90000000,45.647059
2021-02-03 04:00:00,12.66200000,12.78600000,12.65700000,12.78300000,75113.59500000,50.875184
2021-02-03 04:15:00,12.78500000,12.90400000,12.78000000,12.79600000,88620.23200000,51.434164
2021-02-03 04:30:00,12.79600000,12.87500000,12.76600000,12.82900000,48473.90700000,52.899299


In [28]:
import matplotlib
import mplfinance as mpf
dohlc = dohlc.astype({"Open": float, "High": float,"Low": float, "Close": float,"Volume":float
})
dohlc.to_csv('./files/dohlc.csv')
#mpf.plot(dohlc,type='ohlc')

NameError: name 'dohlc' is not defined

In [23]:
type(dohlc.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [26]:
ohlcr = pd.read_csv('C:/Proje/bb/files/hadi.csv')
final_date=[]
date = ohlcr['Opem Time']
for time in date.unique():
    time = time/1000
    readable = datetime.utcfromtimestamp(time)
    final_date.append(readable)
ohlcr.set_index(pd.DatetimeIndex(final_date), inplace=True)

In [31]:
ohlcr = ohlcr.drop('Opem Time',axis=1)

In [27]:
ohlcr.index.name = 'Date'


NameError: name 'ohlcr' is not defined

In [26]:
tl = ohlcr.loc['2021-02-03 13:30':'2021-02-03 21:15:00'].copy()
ll = bb.positive_mismatch(ohlc8)


NameError: name 'ohlcr' is not defined

In [25]:
for i in range(1,int(len(ohlcr)/32 +1)):
    ohlc8 = ohlcr.iloc[(i):(32*1)].copy()
    isPositive = bb.positive_mismatch(ohlc8)
 

NameError: name 'ohlcr' is not defined

In [15]:
klines = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_15MINUTE, "1 day ago UTC")

In [16]:
#color =  bb.COC(klines)
color = pd.DataFrame(klines, columns=config.FIELD_HISTORICAL)
color.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,QA Volume,Number of Trades,TBBA Volume,TBQA Volume,Ignore
0,1613896200000,56099.98000000,56444.26000000,56095.67000000,56361.37000000,477.01136400,1613897099999,26840417.44027420,14312,251.86924600,14171131.79793705,0
1,1613897100000,56361.36000000,56676.70000000,56358.87000000,56513.22000000,636.85460700,1613897999999,36003960.92696925,17864,382.10512000,21600964.80649716,0
2,1613898000000,56513.21000000,56610.00000000,56400.00000000,56424.12000000,341.98260000,1613898899999,19320539.13270400,13749,163.33173300,9227816.28584092,0
3,1613898900000,56424.12000000,56547.82000000,56388.69000000,56450.98000000,416.04004600,1613899799999,23492906.75163787,14337,233.24785800,13170855.79008158,0
4,1613899800000,56450.99000000,56500.00000000,56367.25000000,56417.52000000,387.21904200,1613900699999,21851754.81032528,14581,208.91278500,11789033.29289153,0


In [17]:
def set_date(df):
    date = df['Open Time']
    final_date = []
    for time in date.unique():
        time = time/1000
        readable = datetime.utcfromtimestamp(time)
        final_date.append(readable) 
    df.set_index(pd.DatetimeIndex(final_date), inplace=True)


In [18]:
set_date(color)
rsi = talib.RSI(color['Close'],14)
color['RSI'] = rsi
color.dropna(inplace=True)
color = color[['Open','High','Low','Close','Volume','RSI']]
color.head()
color.sort_index(ascending=False, inplace=True)
 

In [27]:
def COCD(df):
    color= []
    for i in range(len(df)):
       
        if float(df.iloc[i,0]) > float(df.iloc[i,3]):
            color.append('RED')
        else:
            color.append('GREEN')
    df['C'] = color
    



In [8]:
str(color.iloc[0].name)

'2021-02-22 08:00:00'

In [9]:
color_grp = color.groupby('C')

In [10]:
color_grp.get_group('RED').head()

,Open,High,Low,Close,Volume,RSI,C
2021-02-22 08:00:00,56422.23000000,56623.69000000,56282.54000000,56302.03000000,476.89642700,48.880971,RED
2021-02-22 07:45:00,56533.62000000,56550.00000000,56267.04000000,56422.92000000,605.08954800,51.181223,RED
2021-02-22 06:45:00,56203.11000000,56350.00000000,55989.35000000,55989.36000000,459.60815800,42.933170,RED
2021-02-22 06:30:00,56226.37000000,56300.00000000,56004.35000000,56203.11000000,406.76438200,46.321209,RED
2021-02-22 06:15:00,56289.19000000,56377.00000000,56162.76000000,56223.24000000,450.69291900,46.643090,RED


In [12]:
def get_signal(df):
    date = str(df.iloc[0].name)
    signals= pd.DataFrame(columns=[date,'Close>','RSI>','Close','RSI'])
  
    
    close = float(df.iloc[0,3])
    rsi = float(df.iloc[0,5])
    for i in range(1,(len(df))):
        if close > float(df.iloc[i,3]):
            isBTC  = True
        else:
            isBTC = False
            
        if rsi > float(df.iloc[i,5]):
            isBTR  = True
        else:
            isBTR = False
        
        if isBTC != isBTR:
            signals = signals.append({date:df.iloc[i].name,'Close>':isBTC,'RSI>':isBTR,'Close':df.iloc[i,3],'RSI':df.iloc[i,5]},ignore_index=True)
    
    
    signals = signals.set_index(signals[date]).drop(date, 1 )
    return signals

In [13]:
if str(color.iloc[0,6]) == 'GREEN':
    signal = get_signal(color_grp.get_group('GREEN'))
else:
    signal = get_signal(color_grp.get_group('RED'))


In [25]:
c = color.tail(1)['C'].values[0]

In [4]:
#color =  bb.COC(klines)
klines = client.get_historical_klines("SUSHIUSDT", Client.KLINE_INTERVAL_15MINUTE, "1 day ago UTC")
df1 = pd.DataFrame(klines, columns=config.FIELD_HISTORICAL)
bb.set_date(df1)
df1 = df1[['Open','High','Low','Close','Volume']].copy()
df1.head()
high = df1['High']
low = df1['Low']
close = df1['Close']
volume = df1['Volume']


In [5]:
rsi = RSI(close,14)
ndi = MINUS_DI(high, low, close, timeperiod=14)
pdi = PLUS_DI(high, low, close, timeperiod=14)
adx= ADX(high, low, close, timeperiod=14)
atr = ATR(high, low, close, timeperiod=14)

In [6]:
datar = df1.copy()
datar['RSI']= rsi
datar['DI+']= pdi
datar['DI-']= ndi
datar['ADX']= adx
datar['ATR']= atr
datar.dropna(inplace=True)
datar.sort_index(ascending=False, inplace=True)

In [7]:
datar.iloc[0]

Open         15.88900000
High         16.07200000
Low          15.88800000
Close        16.04200000
Volume    21809.09600000
RSI              41.3322
DI+              12.9344
DI-              33.8747
ADX              42.6182
ATR             0.167546
Name: 2021-04-17 13:30:00, dtype: object

In [8]:
pdi

2021-04-16 13:30:00         NaN
2021-04-16 13:45:00         NaN
2021-04-16 14:00:00         NaN
2021-04-16 14:15:00         NaN
2021-04-16 14:30:00         NaN
                         ...   
2021-04-17 12:15:00    9.584332
2021-04-17 12:30:00    8.780015
2021-04-17 12:45:00    8.999223
2021-04-17 13:00:00    8.946861
2021-04-17 13:15:00    8.105226
Length: 96, dtype: float64

In [35]:
adxr

2021-02-22 11:45:00          NaN
2021-02-22 12:00:00          NaN
2021-02-22 12:15:00          NaN
2021-02-22 12:30:00          NaN
2021-02-22 12:45:00          NaN
                         ...    
2021-02-23 10:30:00    42.375055
2021-02-23 10:45:00    42.545208
2021-02-23 11:00:00    42.517205
2021-02-23 11:15:00    43.161397
2021-02-23 11:30:00    43.870719
Length: 96, dtype: float64

In [36]:
atr

2021-02-22 11:45:00         NaN
2021-02-22 12:00:00         NaN
2021-02-22 12:15:00         NaN
2021-02-22 12:30:00         NaN
2021-02-22 12:45:00         NaN
                         ...   
2021-02-23 10:30:00    0.700923
2021-02-23 10:45:00    0.680857
2021-02-23 11:00:00    0.656153
2021-02-23 11:15:00    0.665285
2021-02-23 11:30:00    0.648479
Length: 96, dtype: float64

In [44]:
deneme = pd.DataFrame([['cl','12']],columns=['d1','d2'])
deneme.to_csv('omer.csv')
deneme

,d1,d2
0,cl,12


In [62]:
la = pd.DataFrame([['sa2','2']],columns=['d1','d2'])

la


,d1,d2
0,sa2,2


In [63]:

import os
if os.path.exists('omer1.csv'):
    la.to_csv('omer1.csv',mode='a',header=False)
else:
    la.to_csv('omer1.csv',mode='a')

In [85]:
class position:
  
    def __init__(self,df,symbol,margin,leverage,sl):
        
        self.open_price = float(df.iloc[3])
        self.symbol = symbol
        self.rsi = float(df.iloc[5])
        self.pdi = float(df.iloc[6])
        self.ndi = float(df.iloc[7])
        self.adx = float(df.iloc[8])
        self.atr = float(df.iloc[9])
        self.margin = margin
        self.leverage = leverage
        self.date = df.name
        self.stop_loss = self.open_price - self.atr*2
        self.sl = sl
        
        
        print('{}_{} : Proccess is opened'.format(self.date,symbol))
        df_open = pd.DataFrame([[self.sl,self.date,self.open_price,self.stop_loss,self.margin,self.leverage]],columns=['Position','Open Time','Open Price','Stop Loss','Margin','Leverage'])
        print(df_open)
      
        
    def close_position(self,success):
        rate = (self.current_price - self.open_price) / self.open_price
        pl = rate * self.margin * self.leverage
        
        df_close = pd.DataFrame([[self.symbol,self.sl,self.date,self.current_date,self.open_price,self.current_price,rate,success,self.margin,self.leverage,pl]],columns=['Symbol','Position','Open Time','Close Time','Open Price','Close Price','Rate','Success','Margin','Leverage','PL'])
        if os.path.exists('Trades.csv'):
            df_close.to_csv('Trades.csv',mode='a',header=False)
        else:
            df_close.to_csv('Trades.csv',mode='a')
            
        print(df_close)
       
    
    def get_price_time(self,price,date):
        self.current_price = float(price)
        self.current_date = date
    
        
        
    def __del__(self):
        print('position is closed')
            
    def check_stop_loss(self):
        if self.current_price <= self.stop_loss:
            success = False
            self.close_position(success)
            return True
        return False
        
    def check_dmi_pattern(self,pdi,ndi,adx):
            if ndi>adx>pdi:
                success = False
                if self.current_price >= self.open_price:
                    success = True
               
                self.close_position(success)
                return True
            return False
        
        
        
  

    
        
        
        
        
        
    
    

In [88]:
p1 = position(datar.iloc[0],'SXPUSDT',100,10,'Short')
p1.open_price

2021-04-17 13:30:00_SXPUSDT : Proccess is opened
  Position           Open Time  Open Price  Stop Loss  Margin  Leverage
0    Short 2021-04-17 13:30:00      16.042  15.706908     100        10


16.042

In [89]:
p1.get_price_time(12.084,'2021-02-23 17:15:00')

In [90]:
is_closed = p1.check_stop_loss()

    Symbol Position           Open Time           Close Time  Open Price  \
0  SXPUSDT    Short 2021-04-17 13:30:00  2021-02-23 17:15:00      16.042   

   Close Price      Rate  Success  Margin  Leverage          PL  
0       12.084 -0.246727    False     100        10 -246.727341  


In [91]:
if is_closed:
    del p1

position is closed


In [49]:
print(p1.pdi,
p1.ndi,
p1.adx)

12.93444653540628 33.8746816772088 42.618166290942675


In [50]:
if p1.check_dmi_pattern(8,22,12):
    del p1

      Symbol           Open Time           Close Time  Open Price  \
0  SUSHIUSDT 2021-04-17 13:30:00  2021-02-23 17:15:00      16.042   

   Close Price  Rate  Success  Margin  Leverage      PL  
0       32.084   1.0     True     100        10  1000.0  
position is closed


In [76]:
long_positions[0].open_price

16.042

In [87]:
transactions = []


In [95]:
transactions.append(position(datar.iloc[0],'SUSHIUSDT'))
transactions.append(position(datar.iloc[0],'TOMOUSDT'))
transactions.append(position(datar.iloc[0],'SXPUSDT'))
transactions.append(position(datar.iloc[0],'SUSHIUSDT'))

2021-04-17 09:15:00_SUSHIUSDT : Proccess is opened
            Open Time  Open Price  Stop Loss
0 2021-04-17 09:15:00      16.464  16.195335
2021-04-17 09:15:00_TOMOUSDT : Proccess is opened
            Open Time  Open Price  Stop Loss
0 2021-04-17 09:15:00      16.464  16.195335
2021-04-17 09:15:00_SXPUSDT : Proccess is opened
            Open Time  Open Price  Stop Loss
0 2021-04-17 09:15:00      16.464  16.195335
2021-04-17 09:15:00_SUSHIUSDT : Proccess is opened
            Open Time  Open Price  Stop Loss
0 2021-04-17 09:15:00      16.464  16.195335


In [99]:
len(transactions)

4

In [14]:
long_positions = []
short_positions = []
TRADE_SYMBOL = 'BTCUSDT'

In [97]:
for transaction in transactions:
    transaction.get_price_time(15,'2021-02-23 17:15:00')

position is closed


In [98]:
transactions[0]


In [100]:
for transaction in transactions:
    if transaction.check_stop_loss():
        del transaction
    else:
        print('proccess still opend')


      Symbol           Open Time           Close Time  Open Price  \
0  SUSHIUSDT 2021-04-17 09:15:00  2021-02-23 17:15:00      16.464   

   Close Price      Rate  Success  
0         15.0 -0.088921    False  
     Symbol           Open Time           Close Time  Open Price  Close Price  \
0  TOMOUSDT 2021-04-17 09:15:00  2021-02-23 17:15:00      16.464         15.0   

       Rate  Success  
0 -0.088921    False  
    Symbol           Open Time           Close Time  Open Price  Close Price  \
0  SXPUSDT 2021-04-17 09:15:00  2021-02-23 17:15:00      16.464         15.0   

       Rate  Success  
0 -0.088921    False  
      Symbol           Open Time           Close Time  Open Price  \
0  SUSHIUSDT 2021-04-17 09:15:00  2021-02-23 17:15:00      16.464   

   Close Price      Rate  Success  
0         15.0 -0.088921    False  


In [102]:
i = 0

while i< len(transactions):
    if transactions[i].check_stop_loss():
        del transactions[i]
      
    else:
        print('proccess still opend')
        i +=1


      Symbol           Open Time           Close Time  Open Price  \
0  SUSHIUSDT 2021-04-17 09:15:00  2021-02-23 17:15:00      16.464   

   Close Price      Rate  Success  
0         15.0 -0.088921    False  
     Symbol           Open Time           Close Time  Open Price  Close Price  \
0  TOMOUSDT 2021-04-17 09:15:00  2021-02-23 17:15:00      16.464         15.0   

       Rate  Success  
0 -0.088921    False  
position is closed
    Symbol           Open Time           Close Time  Open Price  Close Price  \
0  SXPUSDT 2021-04-17 09:15:00  2021-02-23 17:15:00      16.464         15.0   

       Rate  Success  
0 -0.088921    False  
position is closed
      Symbol           Open Time           Close Time  Open Price  \
0  SUSHIUSDT 2021-04-17 09:15:00  2021-02-23 17:15:00      16.464   

   Close Price      Rate  Success  
0         15.0 -0.088921    False  
position is closed


In [103]:
len(transactions)

0

In [118]:
p  = [1,2,3,4]

In [109]:
if p[1]:
    print('la')

la


In [1]:

import tests
    
    
global fh 
fh = 'la'

In [3]:



tests.ss()


lalala


In [14]:

klines = client.get_historical_klines('SUSHIUSDT', Client.KLINE_INTERVAL_15MINUTE, "1 day ago UTC")

data = pd.DataFrame(klines, columns=config.FIELD_HISTORICAL)
bb.set_date(data)
data = data[['Open','High','Low','Close','Volume']].copy()
go = data.head()
go

NameError: name 'client' is not defined

In [ ]:
for date, row in go.T.iteritems():
    
    print(type(date))


In [49]:
def isSup(df,i):
    support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['High'][i+1] 
    return support
def isSupR(df,i):
    support = df['RSI'][i] < df['RSI'][i-1]  and df['RSI'][i] < df['RSI'][i+1] 
    return support
def isResR(df,i):
    res = df['RSI'][i] > df['RSI'][i-1]  and df['RSI'][i] > df['RSI'][i+1] 
    return res

In [51]:
levels = []
for i in range(1,data.shape[0]-1):
    if isSup(data,i):
        levels.append((i,data.iloc[i].name))

In [52]:
levels


[(1, Timestamp('2021-04-16 08:45:00')),
 (2, Timestamp('2021-04-16 09:00:00')),
 (3, Timestamp('2021-04-16 09:15:00')),
 (4, Timestamp('2021-04-16 09:30:00')),
 (5, Timestamp('2021-04-16 09:45:00')),
 (8, Timestamp('2021-04-16 10:30:00')),
 (9, Timestamp('2021-04-16 10:45:00')),
 (10, Timestamp('2021-04-16 11:00:00')),
 (11, Timestamp('2021-04-16 11:15:00')),
 (15, Timestamp('2021-04-16 12:15:00')),
 (17, Timestamp('2021-04-16 12:45:00')),
 (18, Timestamp('2021-04-16 13:00:00')),
 (21, Timestamp('2021-04-16 13:45:00')),
 (26, Timestamp('2021-04-16 15:00:00')),
 (28, Timestamp('2021-04-16 15:30:00')),
 (30, Timestamp('2021-04-16 16:00:00')),
 (31, Timestamp('2021-04-16 16:15:00')),
 (32, Timestamp('2021-04-16 16:30:00')),
 (34, Timestamp('2021-04-16 17:00:00')),
 (37, Timestamp('2021-04-16 17:45:00')),
 (38, Timestamp('2021-04-16 18:00:00')),
 (39, Timestamp('2021-04-16 18:15:00')),
 (40, Timestamp('2021-04-16 18:30:00')),
 (43, Timestamp('2021-04-16 19:15:00')),
 (51, Timestamp('2021-0

In [53]:
rsi = RSI(data['Close'])


In [54]:
data['RSI'] = rsi

In [55]:
data = data.dropna()

In [56]:
data.head()

,Open,High,Low,Close,Volume,RSI
2021-04-16 12:00:00,16.34900000,16.35100000,16.21500000,16.27700000,55898.78500000,30.923451
2021-04-16 12:15:00,16.27400000,16.36000000,16.12100000,16.34200000,23702.98900000,33.741259
2021-04-16 12:30:00,16.34900000,16.39400000,16.34100000,16.37300000,27169.08900000,35.100986
2021-04-16 12:45:00,16.37200000,16.37900000,16.23000000,16.23200000,23548.00400000,31.894921
2021-04-16 13:00:00,16.23200000,16.34100000,16.12200000,16.31400000,42601.89900000,35.580051


In [57]:
levels = []
for i in range(2,data.shape[0]-2):
    if isSup(data,i):
        levels.append((data.iloc[i].name))

In [58]:
rsis = []
for i in range(1,data.shape[0]-1):
    if isResR(data,i):
        rsis.append((data.iloc[i].name))

In [44]:
if levels[0] in rsis:
    print('la la la')

In [59]:
rsis

[Timestamp('2021-04-16 12:30:00'),
 Timestamp('2021-04-16 13:00:00'),
 Timestamp('2021-04-16 14:30:00'),
 Timestamp('2021-04-16 16:00:00'),
 Timestamp('2021-04-16 16:45:00'),
 Timestamp('2021-04-16 17:30:00'),
 Timestamp('2021-04-16 19:00:00'),
 Timestamp('2021-04-16 20:15:00'),
 Timestamp('2021-04-16 20:45:00'),
 Timestamp('2021-04-16 21:30:00'),
 Timestamp('2021-04-16 23:00:00'),
 Timestamp('2021-04-17 00:00:00'),
 Timestamp('2021-04-17 01:00:00'),
 Timestamp('2021-04-17 01:30:00'),
 Timestamp('2021-04-17 02:00:00'),
 Timestamp('2021-04-17 02:45:00'),
 Timestamp('2021-04-17 03:45:00'),
 Timestamp('2021-04-17 04:45:00'),
 Timestamp('2021-04-17 05:45:00'),
 Timestamp('2021-04-17 06:15:00'),
 Timestamp('2021-04-17 07:30:00')]

In [32]:
result = set(rsis).intersection(levels)

In [33]:
result

{Timestamp('2021-04-16 12:45:00'),
 Timestamp('2021-04-16 13:45:00'),
 Timestamp('2021-04-16 16:15:00'),
 Timestamp('2021-04-16 17:00:00'),
 Timestamp('2021-04-16 18:15:00'),
 Timestamp('2021-04-16 19:15:00'),
 Timestamp('2021-04-16 21:15:00'),
 Timestamp('2021-04-16 22:45:00'),
 Timestamp('2021-04-17 00:30:00'),
 Timestamp('2021-04-17 02:15:00'),
 Timestamp('2021-04-17 03:30:00'),
 Timestamp('2021-04-17 04:30:00'),
 Timestamp('2021-04-17 06:45:00')}

In [138]:
data

,Open,High,Low,Close,Volume,RSI
2021-04-13 17:15:00,15.80700000,15.94300000,15.80700000,15.88600000,63271.26700000,54.619970
2021-04-13 17:30:00,15.90000000,15.91800000,15.77800000,15.88700000,87164.77400000,54.656357
2021-04-13 17:45:00,15.88700000,15.89600000,15.75900000,15.82300000,35573.97400000,51.793975
2021-04-13 18:00:00,15.81400000,15.88100000,15.80300000,15.87700000,29404.88800000,53.983736
2021-04-13 18:15:00,15.87600000,15.90500000,15.81800000,15.86300000,34008.09500000,53.307647
...,...,...,...,...,...,...
2021-04-14 12:30:00,16.62200000,16.62700000,16.42700000,16.48800000,99203.88800000,45.413594
2021-04-14 12:45:00,16.49100000,16.51100000,16.27000000,16.28300000,63077.43700000,39.821892
2021-04-14 13:00:00,16.28400000,16.51600000,16.25500000,16.37700000,55765.83400000,43.271113
2021-04-14 13:15:00,16.38800000,16.52500000,16.37100000,16.38000000,45664.80300000,43.382648
